# Background
This notebook goes through the off position reduction in class for the orion B map

In [1]:
from IPython.display import HTML

HTML('''
// http://lokeshdhakar.com/projects/lightbox2
<link href="./notebook_support/lightbox2-master/dist/css/lightbox.css" rel="stylesheet">

<script src="./notebook_support/lightbox2-master/dist/js/lightbox.js"></script>

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# import commands

In [2]:
## import
from astropy.io import fits
import numpy as np
import glob
import datetime
import pandas as pd
import logging
import os,re
import time
import subprocess
from matplotlib.lines import lineStyles
import jinja2

# notebook
from IPython.display import Image, display
from IPython.display import HTML
# astropy
from astropy.io import fits
from astropy import constants as const
from astropy import units as u


def PrintMessage(message):
    html='''
    <table style="width:100%">
        <tr>
            <td style='text-align:center;vertical-align:middle'>{0}</td>
        </tr>
    </table>
    '''
    display(HTML(html.format(message)))


def Change1dListTo2dList(array, column=2):
    test = []
    test_row = []
    for item in array:
        test_row.append(item)
        if len(test_row)> column-1:
            test.append(test_row)
            test_row = []
    if len(test_row)>0:
        test.append(test_row)
    return test

def DisplayImagesInColumns(images,image_columns=2, gallery_label=""):
    popup_css='''    
    
    <style>
    .thumb img { 
        border:1px solid #000;
        margin:3px;    
        float:left;
    }
    .thumb span { 
        position:fixed;
        visibility:hidden;
    }
    .thumb:hover, .thumb:hover span { 
        visibility:visible;
        z-index:1;
    }

    </style>

    '''
    
    html='''    
    <a href={0} target="_blank">
    <img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    </a>
    '''
    html='''    
    <a href={0}  target="_blank">
    <img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    </a>
    '''
    
    html='''    
    <a href={0}  data-lightbox={3} data-title="{0}">
    <img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    </a>
    '''
    
    list2d = Change1dListTo2dList(images, column=image_columns)
    width_percent = (1/float(image_columns))*100.0
    gallery_label = gallery_label.replace(" ","")
    for row in list2d:
        imagesList=popup_css.join( [html.format(image,width_percent-2,time.time(),gallery_label) 
                     for image in row ])
        display(HTML(imagesList))
    
# Helper Class to run class_cmd_strings and export the images that were created to the notebook
class RunClassScript():
    def __init__(self, cmd, variables=False, no_output=False, display_images=True, test=False):
        if variables:
            template = jinja2.Template(cmd)
            cmd = template.render(variables)
        self.cmd = cmd
        self.cmd_file = "/tmp/cmd.class"
        self.images = [string.split()[1].replace("\"","").replace("//","/") for string in self.cmd.split("\n") if string[0:2] == "ha"]
        if self.cmd.split("\n")[-1].strip() != "exit 0":
            self.cmd = "{}\nexit 0\n".format(self.cmd)
        subprocess.call(["rm", "-rf", "tmp/cmd.class"])
        fileout = open(self.cmd_file, "w")
        fileout.write(self.cmd)
        fileout.close()
        #output = subprocess.check_output("class -nw @{0}".format(self.cmd_file), shell=True)
        if test:
            print self.cmd            
        output = subprocess.check_call("class -nw @{0}".format(self.cmd_file), shell=True)
        if output!=0:
            print output
        #if not no_output:
        #    print output[820:]
        if display_images:
            for image in self.images:
                if os.path.exists(image):
                    display(image, Image(image),width=300,height=300)
                    
def GenerateGIFAnimation(images,gif_filename):
    convert_cmd = "convert -delay 50"\
                  " -loop 0 "\
                  "{0} "\
                  "{1}".format(images,gif_filename)
    print convert_cmd
    os.system(convert_cmd)
    

    from IPython.display import HTML

# Generate overview plots
Here is an example of simple script to plot a single class spectra and display this in the notebook

In [3]:
cmd='''
file in "{{ input_path}}/{{ filename }}"
set unit v
set mode x {{ min_x }} {{ max_x }}
set mode y {{ min_y }} {{ max_y }}
set line {{ line }}
set source {{ source }}
set window {{ window }}

find
get f
smooth box {{ smooth }}
pl
draw text 4 14 'source'


ha "./plots/pixel_{{ source }}_H_V.png" /dev png /overwrite

'''


In [6]:
class_cmd = {}
class_cmd['input_path'] = os.path.abspath("./test_data")
class_cmd['filename'] = "test_spectra.kosma"
class_cmd['min_x'] = -40
class_cmd['max_x'] = 40
class_cmd['min_y'] = -30
class_cmd['max_y'] = 30
class_cmd['smooth'] = 10
class_cmd['line'] = "CII_L"
class_cmd['source'] = "OMC_CENTER"
class_cmd['window'] = "-20 20"

images = []
for i in range(3):
    output = RunClassScript(cmd, variables=class_cmd, no_output=False,display_images=False)
    images.extend(output.images)
message = "Test message"
PrintMessage(message)
DisplayImagesInColumns(images,3,'test_gallery')

Test message
